# Sistema de Procesamiento de Datos de Ventas

Este notebook procesa datos de ventas desde un archivo CSV, los almacena en SQLite, realiza limpieza de datos y genera resúmenes mensuales por región.

## Características
- Carga de datos desde CSV a SQLite
- Eliminación de duplicados y registros nulos
- Cálculo de resumen de ventas por región y mes
- Exportación a CSV del resumen procesado

---

## Paso 1: Clonar repositorio e instalar dependencias

In [ ]:
!git clone https://github.com/Thomassemino/Prueba-Tecnica-Anepsa.git
%cd Prueba-Tecnica-Anepsa
!pip install -q -r requirements.txt

## Paso 2: Verificar datos de entrada

In [ ]:
import pandas as pd
from pathlib import Path

archivo_csv = "Archivo ventas_simuladas.csv"
df = pd.read_csv(archivo_csv)

print(f"Total de registros en el CSV: {len(df)}")
print(f"\nPrimeras 10 filas:")
display(df.head(10))

print(f"\nInformación del dataset:")
print(df.info())

print(f"\nEstadísticas descriptivas:")
display(df.describe())

## Paso 3: Ejecutar procesamiento de datos

In [ ]:
!python procesar_ventas.py

## Paso 4: Verificar resultados

In [ ]:
archivo_resumen = "resumen_ventas_mensual.csv"
df_resumen = pd.read_csv(archivo_resumen)

print(f"Total de registros en el resumen: {len(df_resumen)}")
print(f"\nResumen completo:")
display(df_resumen)

print(f"\nTotal de ventas por región:")
ventas_por_region = df_resumen.groupby('region')['total_ventas'].sum().sort_values(ascending=False)
display(ventas_por_region)

## Paso 5: Visualizar datos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
ventas_por_region = df_resumen.groupby('region')['total_ventas'].sum().sort_values(ascending=False)
ventas_por_region.plot(kind='bar', color='skyblue')
plt.title('Total de Ventas por Región', fontsize=14, fontweight='bold')
plt.xlabel('Región')
plt.ylabel('Total Ventas')
plt.xticks(rotation=45)

plt.subplot(2, 2, 2)
df_resumen['año_mes'] = df_resumen['año'].astype(str) + '-' + df_resumen['mes'].astype(str).str.zfill(2)
ventas_por_mes = df_resumen.groupby('año_mes')['total_ventas'].sum()
ventas_por_mes.plot(kind='line', marker='o', color='green')
plt.title('Evolución de Ventas por Mes', fontsize=14, fontweight='bold')
plt.xlabel('Mes')
plt.ylabel('Total Ventas')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
pivot_data = df_resumen.pivot_table(values='total_ventas', index='mes', columns='region', aggfunc='sum', fill_value=0)
pivot_data.plot(kind='bar', stacked=True, colormap='Set3')
plt.title('Ventas por Región y Mes (Apilado)', fontsize=14, fontweight='bold')
plt.xlabel('Mes')
plt.ylabel('Total Ventas')
plt.legend(title='Región', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.subplot(2, 2, 4)
ventas_por_region_mes = df_resumen.groupby(['region', 'año_mes'])['total_ventas'].sum().reset_index()
for region in ventas_por_region_mes['region'].unique():
    data = ventas_por_region_mes[ventas_por_region_mes['region'] == region]
    plt.plot(data['año_mes'], data['total_ventas'], marker='o', label=region)
plt.title('Tendencia de Ventas por Región', fontsize=14, fontweight='bold')
plt.xlabel('Mes')
plt.ylabel('Total Ventas')
plt.xticks(rotation=45)
plt.legend(title='Región')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Resumen estadístico:")
print(f"  • Región con mayores ventas: {ventas_por_region.idxmax()} (${ventas_por_region.max():,.2f})")
print(f"  • Región con menores ventas: {ventas_por_region.idxmin()} (${ventas_por_region.min():,.2f})")
print(f"  • Total general de ventas: ${df_resumen['total_ventas'].sum():,.2f}")
print(f"  • Promedio de ventas mensuales: ${df_resumen['total_ventas'].mean():,.2f}")

## Paso 6: Consultar base de datos SQLite directamente

In [ ]:
import sqlite3

conexion = sqlite3.connect('anepsa_ventas.db')

print("📋 Consulta SQL: Top 10 ventas individuales más grandes\n")
query_top_ventas = """
SELECT 
    fecha,
    region,
    producto,
    cantidad,
    precio_unitario,
    (cantidad * precio_unitario) as venta_total
FROM ventas_raw
ORDER BY venta_total DESC
LIMIT 10
"""
df_top_ventas = pd.read_sql_query(query_top_ventas, conexion)
display(df_top_ventas)

print("\n📊 Consulta SQL: Cantidad de ventas por producto\n")
query_productos = """
SELECT 
    producto,
    COUNT(*) as cantidad_transacciones,
    SUM(cantidad) as unidades_vendidas,
    SUM(cantidad * precio_unitario) as total_ventas
FROM ventas_raw
GROUP BY producto
ORDER BY total_ventas DESC
"""
df_productos = pd.read_sql_query(query_productos, conexion)
display(df_productos)

conexion.close()
print("\n✅ Análisis completado exitosamente")

## Paso 7: Descargar archivos generados

Puedes descargar el resumen generado y la base de datos SQLite.

In [ ]:
from google.colab import files

print("Descargando archivos generados...")
files.download('resumen_ventas_mensual.csv')
files.download('anepsa_ventas.db')
print("✅ Archivos descargados exitosamente")